In [1]:
import sys

sys.path.append("..")

%load_ext autoreload
%autoreload complete

In [2]:
from math import ceil
from functools import partial
from pathlib import Path

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import regionmask
from pylipd.lipd import LiPD

from project.io import IntakeESMLoader, save_dataset
from project.plotting import plot_field
from project.units import convert_thetaot_to_ohc
from project.grid import GLOBAL_GRID
from project.util import stack_state, average_annually

In [3]:
loader = IntakeESMLoader(
    "past1000",
    "MRI-ESM2-0",
    ["tas"],
)
ds = loader.load_dataset()
ds

2023-12-11 20:06:18   DEBUG Opening catalog /home/enkf6/dstiller/CMIP6/catalog.json


2023-12-11 20:06:20    INFO Loading dataset from catalog
2023-12-11 20:06:20   DEBUG  - CMIP6Variable(name='tas', id='tas', table='Amon', pl=None, postprocess_fns=[<function mask_poles at 0x7feeb07cb240>])


<xarray.Dataset>
Dimensions:  (time: 12000, lon: 180, lat: 90)
Coordinates:
  * time     (time) object 0850-01-16 12:00:00 ... 1849-12-16 12:00:00
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
Data variables:
    tas      (time, lat, lon) float32 dask.array<chunksize=(5040, 90, 180), meta=np.ndarray>

In [4]:
ds_annual = average_annually(ds)
ds_annual

<xarray.Dataset>
Dimensions:  (lon: 180, lat: 90, time: 999)
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * time     (time) int64 850 851 852 853 854 855 ... 1844 1845 1846 1847 1848
Data variables:
    tas      (time, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>

In [5]:
# ds.to_netcdf("/home/enkf6/dstiller/test_io_output.nc")

In [6]:
pages2k = LiPD()
pages2k.load_from_dir("/home/enkf6/dstiller/obs/Pages2kTemperature2_1_2")

Loading 647 LiPD files


  0%|          | 0/647 [00:00<?, ?it/s]

100%|██████████| 647/647 [00:31<00:00, 20.26it/s]

Loaded..


In [7]:
locs = (
    pages2k.get_all_locations()
    .sample(frac=1, random_state=np.random.default_rng(46234))
    .rename(columns={"geo_meanLat": "lat", "geo_meanLon": "lon"})
    .reset_index(drop=True)
).iloc[:30]
locs["lon"].loc[locs["lon"] < 0] = 360 + locs["lon"].loc[locs["lon"] < 0]
locs

,dataSetName,lat,lon,geo_meanElev
0,Asi-ZhejiangAndFujian.Zhang.1980,25.0000,118.0000,2400.0
1,Asi-MQAXJP.PAGES2k.2013,35.0700,100.3500,700.0
2,Ocn-LaurentianFan.Keigwin.2005,43.4800,305.1300,-3975.0
3,Asi-PAKI015.Esper.2007,35.1700,75.5000,3300.0
4,Asi-CHIN052.Shao.2013,37.4500,97.5300,2060.0
5,Ant-VLG.Bertler.2011,-77.3302,162.5332,625.0
6,Ocn-WestSpitzberg.Bonnet.2010,78.9600,5.8850,-1497.0
7,SAm-CentralAndes6.Villalba.2014,-38.5000,288.5000,1400.0
8,Asi-NEPA025.Krusic.2013,29.5200,82.0300,4000.0
9,Eur-EuropeanAlps.Buentgen.2011,47.0000,10.7000,2050.0


In [25]:
obs = xr.combine_by_coords(
    [
        ds_annual.sel(lat=[locs.lat.iloc[i]], lon=[locs.lon.iloc[i]], method="nearest").stack(
            dict(location=["lon", "lat"])
        )
        for i in range(len(locs.index))
    ]
)
obs

<xarray.Dataset>
Dimensions:   (time: 999, location: 26)
Coordinates:
  * time      (time) int64 850 851 852 853 854 855 ... 1844 1845 1846 1847 1848
  * location  (location) object MultiIndex
  * lon       (location) float64 6.0 10.0 30.0 34.0 ... 246.0 288.0 306.0 356.0
  * lat       (location) float64 79.0 47.0 65.0 29.0 ... 47.0 -39.0 43.0 -75.0
Data variables:
    tas       (time, location) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>

In [16]:
sigma_obs = 1.5

In [17]:
obs["tas"] += np.random.default_rng(52565641).normal(0, sigma_obs, obs["tas"].shape)

(999, 26)

In [9]:
# fig, axs = plt.subplots(
#     1,
#     5,
#     figsize=(16, 4),
#     subplot_kw=dict(projection=ccrs.EqualEarth(central_longitude=198)),
# )

# for i, ax in enumerate(axs):
#     plot_field(ax, ds_annual.isel(time=i)["tas"], n_level=5, rotate_cbar_ticks=True)
#     ax.coastlines()